In [153]:
import pandas as pd
import json
import os
import numpy as np

In [154]:
vg_path = "/gscratch/krishna/shared_data/datasets/vg" 

In [155]:
# load vg scene graphs as a df 
sg = json.load(open(os.path.join(vg_path, "scene_graphs.json")))
sg_df = pd.DataFrame.from_records(sg)
sg_df.head()

,relationships,image_id,objects
0,"[{'synsets': ['along.r.01'], 'predicate': 'ON'...",1,"[{'synsets': ['clock.n.01'], 'h': 339, 'object..."
1,"[{'synsets': ['wear.v.01'], 'predicate': 'wear...",2,"[{'synsets': [], 'h': 103, 'object_id': 5069, ..."
2,"[{'synsets': ['in.r.01'], 'predicate': 'in fro...",3,"[{'synsets': [], 'h': 79, 'object_id': 5091, '..."
3,"[{'synsets': ['have.v.01'], 'predicate': 'has'...",4,"[{'synsets': ['curtain.n.01'], 'h': 300, 'obje..."
4,"[{'synsets': ['along.r.01'], 'predicate': 'ON'...",5,"[{'synsets': ['floor.n.01'], 'h': 108, 'object..."


In [194]:
out_df = pd.read_csv('output_defined.csv')

In [195]:
out_df.head()

,image_id,objects,relationships
0,770,"[{'object_id': 1637131, 'name': 'crosswalk', '...",[]
1,861,"[{'object_id': 1030020, 'name': 'building', 'b...",[]
2,1017,"[{'object_id': 1541014, 'name': 'artwork', 'bb...","[{'subject_id': 3568085, 'object_id': 1541018,..."
3,2748,"[{'object_id': 1034635, 'name': 'trees', 'bbox...","[{'subject_id': 1034645, 'object_id': 1034635,..."
4,3891,"[{'object_id': 4355114, 'name': 'water', 'bbox...","[{'subject_id': 4355161, 'object_id': 4355122,..."


In [158]:
sg_subset = sg_df[sg_df['image_id'].isin(out_df['image_id'])]
print(len(sg_subset))
sg_subset.head()

100


,relationships,image_id,objects
769,"[{'synsets': ['along.r.01'], 'predicate': 'ON'...",770,"[{'synsets': ['crossing.n.05'], 'h': 148, 'obj..."
860,"[{'synsets': ['stand.v.01'], 'predicate': 'sta...",861,"[{'synsets': ['tile.n.01'], 'h': 284, 'object_..."
1016,"[{'synsets': ['look.v.02'], 'predicate': 'look...",1017,"[{'synsets': ['picture.n.01'], 'h': 386, 'obje..."
2747,"[{'synsets': ['next.r.01'], 'predicate': 'next...",2748,"[{'synsets': ['flower.n.01'], 'h': 81, 'object..."
3890,"[{'synsets': [], 'predicate': 'ON', 'relations...",3891,"[{'synsets': [], 'h': 90, 'object_id': 4355110..."


In [196]:
def get_label_attributes(objects):
    obj2attrs = {}
    for i, obj in enumerate(objects):
        if len(obj['names']) == 1:
            obj_name = obj['names'][0]
            if obj_name not in obj2attrs:
                obj2attrs[obj_name] = (obj['attributes'] if 'attributes' in obj else [])
            else:
                obj2attrs[obj_name] += (obj['attributes'] if 'attributes' in obj else [])
    return obj2attrs

def get_label_relationships(relationships):
    objs2rlts = {}
    for i, rlt in enumerate(relationships):
        sub = rlt['subject_id']
        obj = rlt['object_id']
        rlt = rlt['predicate'].lower().strip()
        pair = (sub, obj)
        objs2rlts[pair] = objs2rlts[pair] + [rlt] if pair in objs2rlts else [rlt]
    return objs2rlts

def get_pred_relationships(relationships):
    objs2rlts = {}
    for i, rlt in enumerate(relationships):
        sub = rlt['subject_id']
        obj = rlt['object_id']
        rlts = [r.lower().strip() for r in rlt['relationships']]
        pair = (sub, obj)
        objs2rlts[pair] = objs2rlts[pair] + rlts if pair in objs2rlts else rlts
    return objs2rlts
            
def get_pred_attributes(objects):
    obj2attrs = {}
    for i, obj in enumerate(objects):
        obj2attrs[obj['name']] = obj['attributes']
    return obj2attrs

def calculate_recall_and_precision(obj2labels, obj2preds):
    recalls = []
    precs = []
    f1s = []
    for obj, preds in obj2preds.items():
        labels = obj2labels[obj]
        correct_preds = set(preds).intersection(set(labels))
        # print(len(correct_preds), len(labels))
        # print(len(correct_preds), len(preds), len(labels))

        assert len(labels) > 0 # we made sure to include only cases with gt annotations
        recall = len(correct_preds) / len(labels)
        recalls.append(recall)

        prec = len(correct_preds) / len(preds) if len(preds) > 0 else 0.0
        precs.append(prec)

        f1 = 2 * prec * recall / (prec + recall) if (prec + recall) > 0 else 0.0
        f1s.append(f1)
    print(recalls, precs)
    avg_recall = np.mean(recalls) if len(recalls) > 0 else 0.0
    avg_precision = np.mean(precs) if len(precs) > 0 else 0.0
    avg_f1 = np.mean(f1s) if len(f1s) > 0 else 0.0
    return {'recall': avg_recall, 'precision': avg_precision, 'f1': avg_f1}
# def get_object_ids_to_relationships(relationships):
# def get_label_relationships_of_object1_to_object2(relationships, object1, object2):

In [197]:
all_attr_metrics = {'recall': [], 'precision': [], 'f1': []}
all_rlt_metrics = {'recall': [], 'precision': [], 'f1': []}

for _, row in out_df.iterrows():
    pred_id = row['image_id'] 
    label_row = sg_subset[sg_subset['image_id'] == pred_id]
    
    pred_rlts = eval(row['relationships'])
    pred_objs = eval(row['objects'])
    pred_attrs = get_pred_attributes(pred_objs)
    pred_rlts = get_pred_relationships(pred_rlts)
    # print(pred_rlts)
    
    label_rlts = label_row['relationships'].values[0]
    label_objs = label_row['objects'].values[0]
    label_attrs = get_label_attributes(label_objs) 
    # if len(label_rlts) == 0:
    #     print(label_row)
    label_rlts = get_label_relationships(label_rlts)
    # print(label_rlts)
    
    if len(label_attrs) > 0:
        attr_metrics = calculate_recall_and_precision(label_attrs, pred_attrs)
        all_attr_metrics['recall'].append(attr_metrics['recall'])
        all_attr_metrics['precision'].append(attr_metrics['precision'])
        all_attr_metrics['f1'].append(attr_metrics['f1'])
        print(attr_metrics)
        
    if len(label_rlts) > 0:
        rlt_metrics = calculate_recall_and_precision(label_rlts, pred_rlts)
        all_rlt_metrics['recall'].append(rlt_metrics['recall'])
        all_rlt_metrics['precision'].append(attr_metrics['precision'])
        all_rlt_metrics['f1'].append(attr_metrics['f1'])
        print(rlt_metrics)  
    else:
        print(label_rlts)

[0.0, 1.0, 0.3333333333333333, 0.0] [0.0, 0.3333333333333333, 0.5, 0.0]
{'recall': 0.3333333333333333, 'precision': 0.20833333333333331, 'f1': 0.225}
[] []
{'recall': 0.0, 'precision': 0.0, 'f1': 0.0}
[0.0, 0.3333333333333333, 0.0, 0.047619047619047616, 0.09090909090909091] [0.0, 0.3333333333333333, 0.0, 0.3333333333333333, 0.5]
{'recall': 0.09437229437229437, 'precision': 0.2333333333333333, 'f1': 0.1141025641025641}
[] []
{'recall': 0.0, 'precision': 0.0, 'f1': 0.0}
[0.0, 0.0, 1.0, 1.0, 0.0] [0.0, 0.0, 0.3333333333333333, 0.5, 0.0]
{'recall': 0.4, 'precision': 0.16666666666666666, 'f1': 0.2333333333333333}
[0.0] [0.0]
{'recall': 0.0, 'precision': 0.0, 'f1': 0.0}
[0.0, 0.0, 1.0, 1.0, 0.0] [0.0, 0.0, 0.5, 0.3333333333333333, 0.0]
{'recall': 0.4, 'precision': 0.16666666666666666, 'f1': 0.2333333333333333}
[0.16666666666666666, 1.0, 0.0, 0.5] [0.5, 0.5, 0.0, 1.0]
{'recall': 0.4166666666666667, 'precision': 0.5, 'f1': 0.3958333333333333}
[0.5, 0.08333333333333333, 0.3333333333333333, 0.00

In [198]:
for metric in ['recall', 'precision', 'f1']:
    print(f"{metric} scores:")
    # print(all_attr_metrics[metric])
    print(f"attribute: {np.mean(all_attr_metrics[metric])} (valid examples: {len(all_attr_metrics[metric])}).")
    print(f"relationship: {np.mean(all_rlt_metrics[metric])} (valid examples: {len(all_rlt_metrics[metric])}).")

recall scores:
attribute: 0.17486552234728528 (valid examples: 100).
relationship: 0.22579925128439382 (valid examples: 97).
precision scores:
attribute: 0.11952738095238095 (valid examples: 100).
relationship: 0.12021723122238587 (valid examples: 97).
f1 scores:
attribute: 0.1177291435504072 (valid examples: 100).
relationship: 0.11864565609025779 (valid examples: 97).
